# ADV_DSI_AT2 - Prepare data for training pytorch neural network - baseline model

In [1]:
import pandas as pd
import numpy as np
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

from joblib import dump 
from joblib import load

import torch
import torch.nn as nn
import torch.nn.functional as F

from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader

from scipy.stats import mode
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

## Explore Data

In [2]:
df = pd.read_csv('../data/processed/beer_train_set.csv')

In [3]:
df.head()

,brewery_name,review_aroma,review_appearance,review_palate,review_taste,beer_abv,beer_style
0,Avery Brewing Company,2.5,3.5,3.0,2.5,5.5,American Amber / Red Ale
1,Avery Brewing Company,3.5,4.0,4.0,4.0,5.5,American Amber / Red Ale
2,Avery Brewing Company,4.0,3.5,4.5,5.0,5.5,American Amber / Red Ale
3,Avery Brewing Company,3.5,4.0,3.5,4.0,5.5,American Amber / Red Ale
4,Avery Brewing Company,4.0,3.0,4.0,4.0,5.5,American Amber / Red Ale


In [4]:
df.shape

(7500, 7)

## Prepare Data 1 - Encode target categorical variable to ordinal integer data type

In [5]:
df_cleaned = df.copy()

In [6]:
df_cleaned

,brewery_name,review_aroma,review_appearance,review_palate,review_taste,beer_abv,beer_style
0,Avery Brewing Company,2.5,3.5,3.0,2.5,5.5,American Amber / Red Ale
1,Avery Brewing Company,3.5,4.0,4.0,4.0,5.5,American Amber / Red Ale
2,Avery Brewing Company,4.0,3.5,4.5,5.0,5.5,American Amber / Red Ale
3,Avery Brewing Company,3.5,4.0,3.5,4.0,5.5,American Amber / Red Ale
4,Avery Brewing Company,4.0,3.0,4.0,4.0,5.5,American Amber / Red Ale
...,...,...,...,...,...,...,...
7495,Victory Brewing Company,4.5,4.5,4.5,4.5,9.1,Russian Imperial Stout
7496,Victory Brewing Company,4.0,4.0,4.0,4.0,9.1,Russian Imperial Stout
7497,Victory Brewing Company,4.5,5.0,4.5,4.5,9.1,Russian Imperial Stout
7498,Victory Brewing Company,4.5,4.5,5.0,4.5,9.1,Russian Imperial Stout


In [7]:
# Target variable must be converted to ordinal. Order ategories from most to least frequent, thus beer_style = 0 = American IPA = the mode of the data.
# convert to int

cats = [['American IPA', 'American Double / Imperial IPA', 'Russian Imperial Stout', 'American Pale Ale (APA)', 'American Double / Imperial Stout', 'American Strong Ale', 'American Porter', 'American Amber / Red Ale', 'Belgian Strong Dark Ale', 'Fruit / Vegetable Beer']]

col_encoder = OrdinalEncoder(categories = cats)
df_cleaned['beer_style'] = col_encoder.fit_transform(df_cleaned[['beer_style']])
df_cleaned['beer_style'] = df_cleaned['beer_style'].astype(int)

In [8]:
df_cleaned

,brewery_name,review_aroma,review_appearance,review_palate,review_taste,beer_abv,beer_style
0,Avery Brewing Company,2.5,3.5,3.0,2.5,5.5,7
1,Avery Brewing Company,3.5,4.0,4.0,4.0,5.5,7
2,Avery Brewing Company,4.0,3.5,4.5,5.0,5.5,7
3,Avery Brewing Company,3.5,4.0,3.5,4.0,5.5,7
4,Avery Brewing Company,4.0,3.0,4.0,4.0,5.5,7
...,...,...,...,...,...,...,...
7495,Victory Brewing Company,4.5,4.5,4.5,4.5,9.1,2
7496,Victory Brewing Company,4.0,4.0,4.0,4.0,9.1,2
7497,Victory Brewing Company,4.5,5.0,4.5,4.5,9.1,2
7498,Victory Brewing Company,4.5,4.5,5.0,4.5,9.1,2


## Prepare Data 2 - Split data into train, test and validation sets

In [9]:
# Define a function 

def pop_target(df, target_col, to_numpy=False):
    """Extract target variable from dataframe and convert to nympy arrays if required

    Parameters
    ----------
    df : pd.DataFrame
        Dataframe
    target_col : str
        Name of the target variable
    to_numpy : bool
        Flag stating to convert to numpy array or not

    Returns
    -------
    pd.DataFrame/Numpy array
        Subsetted Pandas dataframe containing all features
    pd.DataFrame/Numpy array
        Subsetted Pandas dataframe containing the target
    """

    df_copy = df.copy()
    target = df_copy.pop(target_col)
    
    if to_numpy:
        df_copy = df_copy.to_numpy()
        target = target.to_numpy()
    
    return df_copy, target

# Define a function to split train test and validation data sets.

def split_sets_random(df, target_col, test_ratio=0.2, to_numpy=False):
    """Split sets randomly

    Parameters
    ----------
    df : pd.DataFrame
        Input dataframe
    target_col : str
        Name of the target column
    test_ratio : float
        Ratio used for the validation and testing sets (default: 0.2)

    Returns
    -------
    Numpy Array
        Features for the training set
    Numpy Array
        Target for the training set
    Numpy Array
        Features for the validation set
    Numpy Array
        Target for the validation set
    Numpy Array
        Features for the testing set
    Numpy Array
        Target for the testing set
    """
    
    from sklearn.model_selection import train_test_split
    
    features, target = pop_target(df=df, target_col=target_col, to_numpy=to_numpy)
    
    X_data, X_test, y_data, y_test = train_test_split(features, target, test_size=test_ratio, random_state=8)
    
    val_ratio = test_ratio / (1 - test_ratio)
    X_train, X_val, y_train, y_val = train_test_split(X_data, y_data, test_size=val_ratio, random_state=8)

    return X_train, y_train, X_val, y_val, X_test, y_test

# Define a function to save train, test ad validation sets to processed data directory. 

def save_sets(X_train=None, y_train=None, X_val=None, y_val=None, X_test=None, y_test=None, path='../data/processed/'):
    """Save the different sets locally

    Parameters
    ----------
    X_train: Numpy Array
        Features for the training set
    y_train: Numpy Array
        Target for the training set
    X_val: Numpy Array
        Features for the validation set
    y_val: Numpy Array
        Target for the validation set
    X_test: Numpy Array
        Features for the testing set
    y_test: Numpy Array
        Target for the testing set
    path : str
        Path to the folder where the sets will be saved (default: '../data/processed/')

    Returns
    -------
    """
    import numpy as np

    if X_train is not None:
      np.save(f'{path}X_train', X_train)
    if X_val is not None:
      np.save(f'{path}X_val',   X_val)
    if X_test is not None:
      np.save(f'{path}X_test',  X_test)
    if y_train is not None:
      np.save(f'{path}y_train', y_train)
    if y_val is not None:
      np.save(f'{path}y_val',   y_val)
    if y_test is not None:
      np.save(f'{path}y_test',  y_test)

In [10]:
# split data into train, test and val sets - do not covert to numpy till final save.

X_train, y_train, X_val, y_val, X_test, y_test = split_sets_random(df_cleaned, target_col='beer_style', test_ratio=0.2, to_numpy=False)

In [11]:
X_train

,brewery_name,review_aroma,review_appearance,review_palate,review_taste,beer_abv
2360,Boston Beer Company (Samuel Adams),4.5,4.0,3.0,2.0,9.2
3815,Founders Brewing Company,4.0,4.5,4.5,4.5,10.5
914,"Bell's Brewery, Inc.",4.5,4.5,4.5,4.5,11.5
7086,Victory Brewing Company,3.5,3.5,3.5,4.5,8.5
135,Avery Brewing Company,5.0,4.5,4.0,4.0,10.3
...,...,...,...,...,...,...
5499,Rogue Ales,4.0,4.5,4.0,4.5,11.0
5074,Rogue Ales,3.0,3.5,3.5,3.0,6.4
1889,Boston Beer Company (Samuel Adams),4.0,4.0,3.5,4.0,6.0
7251,Victory Brewing Company,4.5,4.5,4.5,5.0,5.1


## Prepare Data 3 - Prepare training data

In [12]:
# list categorical variables

cat_cols = ['brewery_name']

In [13]:
X_train.columns

Index(['brewery_name', 'review_aroma', 'review_appearance', 'review_palate',
       'review_taste', 'beer_abv'],
      dtype='object')

In [14]:
# List numerical variables

num_cols = list(set(X_train.columns) - set(cat_cols))
num_cols

['review_aroma',
 'review_appearance',
 'beer_abv',
 'review_palate',
 'review_taste']

In [15]:
X_train[num_cols]

,review_aroma,review_appearance,beer_abv,review_palate,review_taste
2360,4.5,4.0,9.2,3.0,2.0
3815,4.0,4.5,10.5,4.5,4.5
914,4.5,4.5,11.5,4.5,4.5
7086,3.5,3.5,8.5,3.5,4.5
135,5.0,4.5,10.3,4.0,4.0
...,...,...,...,...,...
5499,4.0,4.5,11.0,4.0,4.5
5074,3.0,3.5,6.4,3.5,3.0
1889,4.0,4.0,6.0,3.5,4.0
7251,4.5,4.5,5.1,4.5,5.0


In [16]:
# Use default standard scaler to scale numeric cols.

sc = StandardScaler()
X_train[num_cols] = sc.fit_transform(X_train[num_cols])

In [17]:
X_train[num_cols]

,review_aroma,review_appearance,beer_abv,review_palate,review_taste
2360,0.964149,-0.067906,0.290530,-1.565163,-3.069606
3815,0.133940,0.896064,0.706344,0.971720,0.828911
914,0.964149,0.896064,1.026201,0.971720,0.828911
7086,-0.696268,-1.031877,0.066630,-0.719535,0.828911
135,1.794357,0.896064,0.642373,0.126092,0.049208
...,...,...,...,...,...
5499,0.133940,0.896064,0.866273,0.126092,0.828911
5074,-1.526477,-1.031877,-0.605070,-0.719535,-1.510199
1889,0.133940,-0.067906,-0.733013,-0.719535,0.049208
7251,0.964149,0.896064,-1.020884,0.971720,1.608615


In [18]:
X_train

,brewery_name,review_aroma,review_appearance,review_palate,review_taste,beer_abv
2360,Boston Beer Company (Samuel Adams),0.964149,-0.067906,-1.565163,-3.069606,0.290530
3815,Founders Brewing Company,0.133940,0.896064,0.971720,0.828911,0.706344
914,"Bell's Brewery, Inc.",0.964149,0.896064,0.971720,0.828911,1.026201
7086,Victory Brewing Company,-0.696268,-1.031877,-0.719535,0.828911,0.066630
135,Avery Brewing Company,1.794357,0.896064,0.126092,0.049208,0.642373
...,...,...,...,...,...,...
5499,Rogue Ales,0.133940,0.896064,0.126092,0.828911,0.866273
5074,Rogue Ales,-1.526477,-1.031877,-0.719535,-1.510199,-0.605070
1889,Boston Beer Company (Samuel Adams),0.133940,-0.067906,-0.719535,0.049208,-0.733013
7251,Victory Brewing Company,0.964149,0.896064,0.971720,1.608615,-1.020884


In [19]:
# OneHotEncode brewery_name

ohe = OneHotEncoder(sparse=False)

# OHE
X_cat = pd.DataFrame(ohe.fit_transform(X_train[cat_cols]), index = X_train.index)

# # name OHE'd columns
X_cat.columns = ohe.get_feature_names(cat_cols)

# # drop non-OHE version of column
X_train.drop(cat_cols, axis=1, inplace=True)

# # join training data to new OHE'd columns.
X = pd.concat([X_train, X_cat], axis=1)

X

/opt/conda/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,review_aroma,review_appearance,review_palate,review_taste,beer_abv,brewery_name_Avery Brewing Company,"brewery_name_Bell's Brewery, Inc.",brewery_name_Boston Beer Company (Samuel Adams),brewery_name_Dogfish Head Brewery,brewery_name_Founders Brewing Company,brewery_name_Lagunitas Brewing Company,brewery_name_Rogue Ales,brewery_name_Sierra Nevada Brewing Co.,brewery_name_Stone Brewing Co.,brewery_name_Victory Brewing Company
2360,0.964149,-0.067906,-1.565163,-3.069606,0.290530,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3815,0.133940,0.896064,0.971720,0.828911,0.706344,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
914,0.964149,0.896064,0.971720,0.828911,1.026201,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7086,-0.696268,-1.031877,-0.719535,0.828911,0.066630,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
135,1.794357,0.896064,0.126092,0.049208,0.642373,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5499,0.133940,0.896064,0.126092,0.828911,0.866273,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
5074,-1.526477,-1.031877,-0.719535,-1.510199,-0.605070,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1889,0.133940,-0.067906,-0.719535,0.049208,-0.733013,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7251,0.964149,0.896064,0.971720,1.608615,-1.020884,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


## Prepare Data 4 - build pipeline on train data to apply to test and val data.

In [20]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [21]:
print(cat_cols)
print(num_cols)

['brewery_name']
['review_aroma', 'review_appearance', 'beer_abv', 'review_palate', 'review_taste']


In [22]:
# transformer for numeric cols

num_transformer = Pipeline(
    steps=[
        ('scaler', StandardScaler())
    ]
)

In [23]:
# OHE for categorical cols

cat_transformer = Pipeline(
    steps=[
        ('one_hot_encoder', OneHotEncoder(sparse=False, drop='first'))
    ]
)

In [24]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num_cols', num_transformer, num_cols),
        ('cat_cols', cat_transformer, cat_cols)
    ]
)

In [25]:
dump(preprocessor,  '../models/preprocessor.joblib')

['../models/preprocessor.joblib']

In [26]:
# split data into train, test and val sets - do not covert to numpy till final save.

X_train, y_train, X_val, y_val, X_test, y_test = split_sets_random(df_cleaned, target_col='beer_style', test_ratio=0.2, to_numpy=False)

In [27]:
X_train

,brewery_name,review_aroma,review_appearance,review_palate,review_taste,beer_abv
2360,Boston Beer Company (Samuel Adams),4.5,4.0,3.0,2.0,9.2
3815,Founders Brewing Company,4.0,4.5,4.5,4.5,10.5
914,"Bell's Brewery, Inc.",4.5,4.5,4.5,4.5,11.5
7086,Victory Brewing Company,3.5,3.5,3.5,4.5,8.5
135,Avery Brewing Company,5.0,4.5,4.0,4.0,10.3
...,...,...,...,...,...,...
5499,Rogue Ales,4.0,4.5,4.0,4.5,11.0
5074,Rogue Ales,3.0,3.5,3.5,3.0,6.4
1889,Boston Beer Company (Samuel Adams),4.0,4.0,3.5,4.0,6.0
7251,Victory Brewing Company,4.5,4.5,4.5,5.0,5.1


In [28]:
preprocessor.fit(X_train)

ColumnTransformer(transformers=[('num_cols',
                                 Pipeline(steps=[('scaler', StandardScaler())]),
                                 ['review_aroma', 'review_appearance',
                                  'beer_abv', 'review_palate',
                                  'review_taste']),
                                ('cat_cols',
                                 Pipeline(steps=[('one_hot_encoder',
                                                  OneHotEncoder(drop='first',
                                                                sparse=False))]),
                                 ['brewery_name'])])

In [29]:
X_train = preprocessor.transform(X_train)
X_val = preprocessor.transform(X_val)
X_test = preprocessor.transform(X_test)

In [30]:
X_train

array([[ 0.96414881, -0.06790638,  0.29052976, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.13394031,  0.89606437,  0.70634399, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.96414881,  0.89606437,  1.02620109, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.13394031, -0.06790638, -0.73301296, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.96414881,  0.89606437, -1.02088435, ...,  0.        ,
         0.        ,  1.        ],
       [-0.6962682 , -0.06790638, -0.76499867, ...,  0.        ,
         0.        ,  0.        ]])

In [31]:
# Save transformed train, test and val sets

save_sets(X_train=X_train, y_train=y_train, X_val=X_val, y_val=y_val, X_test=X_test, y_test=y_test, path='../data/processed/')

In [32]:
# define a function to convert daat to pytorch tensors

class PytorchDataset(Dataset):
    """
    Pytorch dataset
    ...

    Attributes
    ----------
    X_tensor : Pytorch tensor
        Features tensor
    y_tensor : Pytorch tensor
        Target tensor

    Methods
    -------
    __getitem__(index)
        Return features and target for a given index
    __len__
        Return the number of observations
    to_tensor(data)
        Convert Pandas Series to Pytorch tensor
    """
        
    def __init__(self, X, y):
        self.X_tensor = self.to_tensor(X)
        self.y_tensor = self.to_tensor(y)
    
    def __getitem__(self, index):
        return self.X_tensor[index], self.y_tensor[index]
        
    def __len__ (self):
        return len(self.X_tensor)
    
    def to_tensor(self, data):
        return torch.Tensor(np.array(data))

In [33]:
train_dataset = PytorchDataset(X=X_train, y=y_train)
val_dataset = PytorchDataset(X=X_val, y=y_val)
test_dataset = PytorchDataset(X=X_test, y=y_test)

In [34]:
X_train.shape[1]

14

## Baseline Model

In [35]:
# For a baseline we will predict every record as the mode of the training data.
# the beer_style categories were passed to ordinal enoder i order of most to least frequent, thus 

mode(y_train)

/tmp/ipykernel_5077/3021002665.py:4: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode(y_train)


ModeResult(mode=array([1]), count=array([630]))

In [36]:
y_base = np.full((len(y_train), 1), 1)

In [37]:
accuracy_score(y_train, y_base)

0.14

In [38]:
f1_score(y_train, y_base, average = 'weighted')

0.0343859649122807

## Define model architecture

In [39]:
class PytorchMultiClass(nn.Module):
    def __init__(self, num_features):
        super(PytorchMultiClass, self).__init__()
        
        self.layer_1 = nn.Linear(num_features, 32)
        self.layer_out = nn.Linear(32, 10)

    def forward(self, x):
        x = F.dropout(F.relu(self.layer_1(x)), training=self.training)
        return self.layer_out(x)

In [40]:
model = PytorchMultiClass(X_train.shape[1])

In [41]:
model

PytorchMultiClass(
  (layer_1): Linear(in_features=14, out_features=32, bias=True)
  (layer_out): Linear(in_features=32, out_features=10, bias=True)
)

In [42]:
def get_device():
    if torch.cuda.is_available():
        device = torch.device('cuda:0')
    else:
        device = torch.device('cpu') # don't have GPU 
    return device

In [43]:
device = get_device()
model.to(device)

PytorchMultiClass(
  (layer_1): Linear(in_features=14, out_features=32, bias=True)
  (layer_out): Linear(in_features=32, out_features=10, bias=True)
)

In [44]:
print(model)

PytorchMultiClass(
  (layer_1): Linear(in_features=14, out_features=32, bias=True)
  (layer_out): Linear(in_features=32, out_features=10, bias=True)
)


## Train Model

In [45]:
criterion = nn.CrossEntropyLoss()

In [46]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

In [47]:
def train_classification(train_data, model, criterion, optimizer, batch_size, device, scheduler=None, generate_batch=None):
    """Train a Pytorch multi-class classification model

    Parameters
    ----------
    train_data : torch.utils.data.Dataset
        Pytorch dataset
    model: torch.nn.Module
        Pytorch Model
    criterion: function
        Loss function
    optimizer: torch.optim
        Optimizer
    bacth_size : int
        Number of observations per batch
    device : str
        Name of the device used for the model
    scheduler : torch.optim.lr_scheduler
        Pytorch Scheduler used for updating learning rate
    collate_fn : function
        Function defining required pre-processing steps

    Returns
    -------
    Float
        Loss score
    Float:
        Accuracy Score
    """
    
    # Set model to training mode
    model.train()
    train_loss = 0
    train_acc = 0
    
    # Create data loader
    data = DataLoader(train_data, batch_size=batch_size, shuffle=True, collate_fn=generate_batch)
    
    # Iterate through data by batch of observations
    for feature, target_class in data:

        # Reset gradients
        optimizer.zero_grad()
        
        # Load data to specified device
        feature, target_class = feature.to(device), target_class.to(device)
        
        # Make predictions
        output = model(feature)
        
        # Calculate loss for given batch
        loss = criterion(output, target_class.long())

        # Calculate global loss
        train_loss += loss.item()
        
        # Calculate gradients
        loss.backward()

        # Update Weights
        optimizer.step()
        
        # Calculate global accuracy
        train_acc += (output.argmax(1) == target_class).sum().item()

    # Adjust the learning rate
    if scheduler:
        scheduler.step()

    return train_loss / len(train_data), train_acc / len(train_data)

In [48]:
def test_classification(test_data, model, criterion, batch_size, device, generate_batch=None):
    """Calculate performance of a Pytorch multi-class classification model

    Parameters
    ----------
    test_data : torch.utils.data.Dataset
        Pytorch dataset
    model: torch.nn.Module
        Pytorch Model
    criterion: function
        Loss function
    bacth_size : int
        Number of observations per batch
    device : str
        Name of the device used for the model
    collate_fn : function
        Function defining required pre-processing steps

    Returns
    -------
    Float
        Loss score
    Float:
        Accuracy Score
    """    
    
    # Set model to evaluation mode
    model.eval()
    test_loss = 0
    test_acc = 0
    
    # Create data loader
    data = DataLoader(test_data, batch_size=batch_size, collate_fn=generate_batch)
    
    # Iterate through data by batch of observations
    for feature, target_class in data:
        
        # Load data to specified device
        feature, target_class = feature.to(device), target_class.to(device)
        
        # Set no update to gradients
        with torch.no_grad():
            
            # Make predictions
            output = model(feature)
            
            # Calculate loss for given batch
            loss = criterion(output, target_class.long())

            # Calculate global loss
            test_loss += loss.item()
            
            # Calculate global accuracy
            test_acc += (output.argmax(1) == target_class).sum().item()

    return test_loss / len(test_data), test_acc / len(test_data)

In [49]:
N_EPOCHS = 50
BATCH_SIZE = 32

In [83]:
# Solution:
# from src.models.pytorch import train_classification, test_classification

for epoch in range(N_EPOCHS):
    train_loss, train_acc = train_classification(train_dataset, model=model, criterion=criterion, optimizer=optimizer, batch_size=BATCH_SIZE, device=device)
    valid_loss, valid_acc = test_classification(test_dataset, model=model, criterion=criterion, batch_size=BATCH_SIZE, device=device)

    print(f'Epoch: {epoch}')
    print(f'\t(train)\t|\tLoss: {train_loss:.4f}\t|\tAcc: {train_acc * 100:.1f}%')
    print(f'\t(valid)\t|\tLoss: {valid_loss:.4f}\t|\tAcc: {valid_acc * 100:.1f}%')

Epoch: 0
	(train)	|	Loss: 0.0570	|	Acc: 28.3%
	(valid)	|	Loss: 0.0498	|	Acc: 34.7%
Epoch: 1
	(train)	|	Loss: 0.0565	|	Acc: 28.4%
	(valid)	|	Loss: 0.0497	|	Acc: 37.9%
Epoch: 2
	(train)	|	Loss: 0.0583	|	Acc: 27.9%
	(valid)	|	Loss: 0.0520	|	Acc: 31.7%
Epoch: 3
	(train)	|	Loss: 0.0595	|	Acc: 26.1%
	(valid)	|	Loss: 0.0514	|	Acc: 36.7%
Epoch: 4
	(train)	|	Loss: 0.0601	|	Acc: 26.7%
	(valid)	|	Loss: 0.0512	|	Acc: 32.9%
Epoch: 5
	(train)	|	Loss: 0.0611	|	Acc: 27.5%
	(valid)	|	Loss: 0.0528	|	Acc: 34.2%
Epoch: 6
	(train)	|	Loss: 0.0597	|	Acc: 26.2%
	(valid)	|	Loss: 0.0539	|	Acc: 29.8%
Epoch: 7
	(train)	|	Loss: 0.0585	|	Acc: 27.0%
	(valid)	|	Loss: 0.0535	|	Acc: 31.5%
Epoch: 8
	(train)	|	Loss: 0.0591	|	Acc: 27.3%
	(valid)	|	Loss: 0.0527	|	Acc: 33.1%
Epoch: 9
	(train)	|	Loss: 0.0594	|	Acc: 26.5%
	(valid)	|	Loss: 0.0526	|	Acc: 29.8%
Epoch: 10
	(train)	|	Loss: 0.0573	|	Acc: 27.0%
	(valid)	|	Loss: 0.0508	|	Acc: 33.9%
Epoch: 11
	(train)	|	Loss: 0.0580	|	Acc: 28.0%
	(valid)	|	Loss: 0.0522	|	Acc: 33.8%
Ep

In [84]:
torch.save(model.state_dict(), "../models/pytorch_beer_classifier.pt")

In [52]:
model.state_dict()

OrderedDict([('layer_1.weight',
              tensor([[-2.5451e-01,  9.4896e-02, -9.9060e-01, -1.6690e-02,  1.3304e-01,
                       -2.4517e-01, -1.8805e+00, -6.7100e-01, -7.1428e-01,  4.9502e-01,
                       -1.3625e+00,  2.3818e-01, -6.9254e-01,  9.0104e-01],
                      [ 3.2977e-01,  1.6602e-01, -6.8277e+00, -3.3532e-01, -2.6006e-01,
                       -1.5230e+00, -1.0201e+00, -1.9545e+00,  1.4953e+00,  1.5864e-01,
                        4.5170e-02, -1.2735e-01, -6.2622e+00,  4.4453e-01],
                      [ 9.0662e-01,  3.5545e-01, -1.7101e-01,  1.9386e-01,  1.4444e-01,
                       -1.7008e+00, -9.4166e-01, -6.0781e-01, -1.7759e+00, -1.4580e+00,
                       -1.0340e+00, -5.8091e-01, -1.8327e+00,  1.2381e+00],
                      [ 1.0468e-01, -1.6777e-01, -7.8947e-02, -1.0421e-02,  1.1269e-01,
                        1.7101e-01, -4.4007e-01, -7.0817e-01,  1.8587e-01, -5.9908e-01,
                       -7.7614e-01, 

In [53]:
model.parameters()

<generator object Module.parameters at 0x7f2f9255db30>

In [85]:
test_loss, test_acc = test_classification(val_dataset, model=model, criterion=criterion, batch_size=BATCH_SIZE, device=device)
print(f'\tLoss: {test_loss:.4f}\t|\tAccuracy: {test_acc:.1f}')

	Loss: 0.0515	|	Accuracy: 0.3


In [55]:
# make predictions on the validation data.

model.eval()
data = DataLoader(val_dataset)
for feature, target_class in data:
        
        # Load data to specified device
        feature, target_class = feature.to(device), target_class.to(device)
        
        # Set no update to gradients
        with torch.no_grad():
            
            # Make predictions
            output = model(feature)

            print(output.argmax(1))

tensor([0])
tensor([3])
tensor([1])
tensor([1])
tensor([3])
tensor([3])
tensor([1])
tensor([1])
tensor([3])
tensor([3])
tensor([1])
tensor([1])
tensor([3])
tensor([1])
tensor([1])
tensor([1])
tensor([1])
tensor([3])
tensor([1])
tensor([1])
tensor([1])
tensor([3])
tensor([1])
tensor([1])
tensor([3])
tensor([1])
tensor([1])
tensor([3])
tensor([0])
tensor([1])
tensor([1])
tensor([1])
tensor([1])
tensor([1])
tensor([1])
tensor([3])
tensor([3])
tensor([1])
tensor([1])
tensor([3])
tensor([1])
tensor([1])
tensor([1])
tensor([3])
tensor([1])
tensor([3])
tensor([3])
tensor([3])
tensor([1])
tensor([3])
tensor([3])
tensor([3])
tensor([3])
tensor([1])
tensor([1])
tensor([1])
tensor([1])
tensor([3])
tensor([1])
tensor([3])
tensor([1])
tensor([1])
tensor([1])
tensor([1])
tensor([1])
tensor([3])
tensor([1])
tensor([0])
tensor([1])
tensor([3])
tensor([1])
tensor([1])
tensor([1])
tensor([3])
tensor([1])
tensor([3])
tensor([1])
tensor([3])
tensor([0])
tensor([0])
tensor([0])
tensor([1])
tensor([3])
tens

In [86]:
model(feature)

tensor([[  1.1134,  -7.9846, -87.3257,   0.5617,  -4.9645,  -2.2732,   1.1192,
           1.0500,  -6.6527,   0.8613]], grad_fn=<AddmmBackward0>)

In [87]:
model(feature).argmax(1)

tensor([6])

In [59]:
from starlette.responses import JSONResponse

In [88]:
model_test = PytorchMultiClass(14)
model_test.load_state_dict(torch.load("../models/pytorch_beer_classifier.pt"))
model_test.eval()

PytorchMultiClass(
  (layer_1): Linear(in_features=14, out_features=32, bias=True)
  (layer_out): Linear(in_features=32, out_features=10, bias=True)
)

In [89]:
def format_features(brewery_name: str,	review_aroma: int, review_appearance: int, review_palate: int, review_taste: int, beer_abv: int):
    return {
        'brewery_name': [brewery_name],
        'review_aroma': [review_aroma],
        'review_appearance': [review_appearance],
        'review_palate': [review_palate],
        'review_taste': [review_taste],
        'beer_abv': [beer_abv]
    }

In [90]:
def predict(brewery_name: str,	review_aroma: int, review_appearance: int, review_palate: int, review_taste: int, beer_abv: int):
    features = format_features(brewery_name, review_aroma, review_appearance, review_palate, review_taste, beer_abv)
    obs = pd.DataFrame(features)
    data = preprocessor.transform(obs)
    data_tensor = torch.Tensor(np.array(data))
    pred = model_test(data_tensor).argmax(1)
    return pred
    # return JSONResponse(pred.tolist())

In [91]:
predict('Boston Beer Company (Samuel Adams)', 4.5, 4.0, 3.0, 2.0, 9.2)

tensor([1])

In [92]:
predict('Avery Brewing Company', 3.5, 4.0, 3.5, 4.0, 5.5)

tensor([7])

In [93]:
predict('Victory Brewing Company', 4.5, 4.5, 4.5, 5.0, 5.1)

tensor([3])

In [94]:
predict('Boston Beer Company (Samuel Adams)', 3.5, 4.0, 2.0, 1.5, 5.9)

tensor([9])

In [95]:
predict('Rogue Ales', 1.0, 5.0, 1.0, 1.0, 5.0)

tensor([9])

In [96]:
predict("Bell's Brewery, Inc.", 4.5, 4.5, 4.5, 4.5, 11.5)

tensor([1])